# A.I. Interview

We want to ask an A.I. expert a series of questions about a particular topic.

### Steps:

1. Create a knowledge base on the topic you'd like to get answers about
2. Do a document review over the knowledge base to generate a list of questions
3. Create a chat thread with access to knowledge bases and web search to let the A.I. expert answer all the questions

In [1]:
import superpowered
import requests
import time
import tqdm

from IPython.display import display, Markdown

SP_BASE_URL = 'https://api.superpowered.ai/v1'
SP_API_KEY_ID = ''
SP_API_KEY_SECRET = ''
API_AUTH = (SP_API_KEY_ID, SP_API_KEY_SECRET)
superpowered.set_api_key(SP_API_KEY_ID, SP_API_KEY_SECRET)


def run_and_display_review(kb: dict, ar_instructions: str, fr_instructions: str):
    url = f'{SP_BASE_URL}/knowledge_bases/{kb["id"]}/review'
    payload = {
        'active_reading_instructions': ar_instructions,
        'final_review_instructions': fr_instructions,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    final_review_display = display(Markdown('Review pending...'), display_id=True)
    progress_bar = tqdm.tqdm(total=1)
    progress_bar.set_description(f'Active Reading Progress for {kb["title"]}')

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        progress_bar.n = resp.json()['response']['active_reading_progress_pct']
        if progress_bar.n >= 1:
            final_review_display.update(Markdown(resp.json()["response"]["final_review"]))
        progress_bar.refresh()

    progress_bar.close()
    return resp.json()['response']['final_review']


def display_chat_question_and_answer(thread_id: str, question: str):
    url = f'{SP_BASE_URL}/chat/threads/{thread_id}/get_response'
    payload = {
        'input': question,
        'async': True,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    answer_display = display(Markdown(f'## {question}\n\n'), display_id=True)

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        answer_display.update(Markdown(f'## {question}\n\n' + str(resp.json()["response"]["interaction"]["model_response"]["content"])))

    return resp.json()['response']


### Generate questions with document review

In [2]:
INTERVIEW_KB_ID = 'e3ed8783-c425-4bfa-99b3-fedba3281048'
INTERVIEW_TOPIC = 'Regenerative Agriculture'


active_reading_instructions = f"""\
Make note of any key points, concepts, or ideas that you think are important related to {INTERVIEW_TOPIC}. \
"""

final_review_instructions = f"""\
Given the notes created during the active reading phase, generate a list of questions regarding {INTERVIEW_TOPIC}.

IMPORTANT: Your output should include nothing except for questions separated by newlines.
"""


kb = superpowered.get_knowledge_base(INTERVIEW_KB_ID)

interview_questions = run_and_display_review(kb, active_reading_instructions, final_review_instructions)

interview_questions = interview_questions.split('\n')
interview_questions = [q for q in interview_questions if q]

What are the core principles of regenerative agriculture?
How do regenerative agriculture practices improve soil health?
What role does biodiversity play in regenerative agriculture?
How does regenerative agriculture contribute to climate change mitigation?
What are the economic benefits for farmers practicing regenerative agriculture?
How do regenerative agriculture practices differ from traditional agricultural methods?
What challenges do farmers face when transitioning to regenerative agriculture practices?
How does regenerative agriculture impact water cycles and water quality?
What are the long-term impacts of regenerative agriculture on land resilience?
How is livestock integrated into regenerative agriculture systems?
What are the specific practices involved in adaptive grazing within regenerative agriculture?
How do regenerative agriculture practices vary across different ecosystems?
What role does carbon sequestration play in regenerative agriculture?
What are the criticisms and limitations of regenerative agriculture?
How do regenerative agriculture practices influence local and global food systems?
What policies and incentives are needed to promote regenerative agriculture on a wider scale?
How do indigenous knowledge and practices influence regenerative agriculture?
What are the key differences between regenerative and organic agriculture?
How do companies incorporate regenerative agriculture into their supply chains?
What scientific evidence supports the benefits of regenerative agriculture?
How do regenerative agriculture practices affect the nutritional quality of food produced?
What role does soil organic matter play in regenerative agriculture?
How does regenerative agriculture address issues of social justice and community involvement?
What are the potential risks of not adopting regenerative agriculture practices globally?
How can regenerative agriculture be scaled up to meet global food demand while maintaining ecosystem health?

Active Reading Progress for Restorative Agriculture (Small): 100%|██████████| 1/1 [01:41<00:00, 101.94s/it]      


### Create a chat thread and start asking the A.I. expert questions

In [3]:
thread = superpowered.create_chat_thread(
    knowledge_base_ids=[INTERVIEW_KB_ID],
    use_web_search=True,
    model='claude-3-opus',
    response_length='short',
    system_message=f'You are acting as an expert in {INTERVIEW_TOPIC}. You should should respond as though the answer is your own. Do NOT refer to any searching you might have done to get the answer. Feel free to reference older messages in the conversation history. Most importantly, you should make your answers accessible to the general public. Do NOT use jargon and other technical terms that are not widely understood.'
)

for question in interview_questions:
    display_chat_question_and_answer(thread['id'], question)


## What are the core principles of regenerative agriculture?

The core principles of regenerative agriculture include improving soil health, enhancing biodiversity, optimizing resource management, mitigating climate change, and improving water quality and availability.

## How do regenerative agriculture practices improve soil health?

Regenerative agriculture practices like minimizing tillage, keeping the soil covered, increasing plant diversity, and integrating livestock help improve soil health by increasing soil organic matter, enhancing soil structure, and supporting a diverse community of beneficial soil microorganisms.

## What role does biodiversity play in regenerative agriculture?

Biodiversity is essential in regenerative agriculture as it enhances ecosystem resilience, improves nutrient cycling, provides natural pest control, and supports a wide range of beneficial organisms both above and below ground.

## How does regenerative agriculture contribute to climate change mitigation?

Regenerative agriculture helps mitigate climate change by sequestering carbon in the soil through practices that increase soil organic matter, reducing greenhouse gas emissions from farming operations, and enhancing the land's resilience to extreme weather events.

## What are the economic benefits for farmers practicing regenerative agriculture?

Regenerative agriculture can boost farm profitability by reducing input costs (like fertilizers and pesticides), improving crop yields and quality over time, enhancing resilience to weather extremes, and potentially opening up new revenue streams like carbon credits or premium pricing for regeneratively-grown products.

...

## How do regenerative agriculture practices affect the nutritional quality of food produced?

Emerging research suggests that regenerative agriculture practices can enhance the nutritional quality of food:

- Some studies have found higher levels of certain vitamins, minerals, and beneficial compounds in crops grown in regenerative systems compared to conventional.
- Livestock raised on regenerative farms with diverse forages may have healthier fat profiles and higher levels of nutrients like omega-3s.
- Regenerative practices can increase the diversity of nutrient-dense foods produced, like fruits, vegetables, and whole grains.

These potential nutritional benefits are thought to result from healthier soils, greater crop diversity, and reduced use of synthetic inputs. However, more research is needed to fully understand the links between specific regenerative practices, soil health, and human health outcomes across different production systems.

## What role does soil organic matter play in regenerative agriculture?

Soil organic matter (SOM) plays a central role in regenerative agriculture. Increasing SOM is a key goal, as it provides numerous benefits:

- Enhances soil structure, aeration, and water-holding capacity
- Acts as a reservoir for nutrients, slowly releasing them to plants
- Supports diverse and abundant soil microbes, which drive nutrient cycling
- Helps buffer soil pH and reduce compaction
- Sequesters carbon from the atmosphere

Regenerative practices like reducing tillage, keeping soil planted, and applying organic amendments all help build SOM over time. SOM is essentially the foundation of soil health in regenerative systems, enabling all the other benefits for crops, water, climate, and biodiversity. Farmers often track SOM as a key indicator of their soil's regenerative capacity.

## How does regenerative agriculture address issues of social justice and community involvement?

Regenerative agriculture has the potential to address social justice issues and promote community involvement, but this aspect is often overlooked. Some key considerations:

- Regenerative practices can improve food security and nutrition for local communities by building more resilient, diverse, and productive food systems.
- Prioritizing Indigenous leadership and traditional ecological knowledge in regenerative transitions is crucial for social justice and decolonizing food systems.
- Regenerative agriculture can create opportunities for more equitable land access and ownership, especially for marginalized groups.
- Community-based initiatives like urban gardens, farmer cooperatives, and local food hubs can make regenerative agriculture more inclusive and accessible.
- Regenerative supply chains should ensure fair labor practices and living wages for farmers and workers.

However, the regenerative agriculture movement has been criticized for often being dominated by privileged, white stakeholders and failing to center social justice. Addressing these disparities and empowering diverse community leadership will be essential for regenerative agriculture to truly advance equity and resilience from the ground up.

## What are the potential risks of not adopting regenerative agriculture practices globally?

If regenerative agriculture practices are not widely adopted, we risk exacerbating multiple global crises:

- Continued soil degradation could lead to widespread crop failures, food insecurity, and economic disruption.
- Biodiversity loss may accelerate as industrial agriculture expands, decimating crucial species and ecosystems.
- Agriculture's greenhouse gas emissions would likely increase, further destabilizing the climate.
- Water cycles and aquatic ecosystems could be increasingly disrupted by erosion, pollution, and unsustainable irrigation.
- Rural communities may face worsening poverty, displacement, and health impacts from industrialized food systems.

While transitioning to regenerative agriculture is complex and challenging, the long-term risks of inaction are severe. Regenerative approaches offer a path to build resilience and avoid catastrophic tipping points in our food systems and ecosystems. The stakes could not be higher.

## How can regenerative agriculture be scaled up to meet global food demand while maintaining ecosystem health?

Scaling up regenerative agriculture to meet global food needs while maintaining ecosystem health will require a multi-faceted approach:

- Increased investment in research, education, and technical assistance to support farmers in adopting regenerative practices
- Redirecting subsidies and policies to incentivize regenerative approaches over industrial agriculture
- Developing markets and infrastructure for regeneratively-grown products, making them more accessible and affordable
- Engaging consumers to shift demand towards regenerative foods and build awareness of their benefits
- Fostering collaborative networks and knowledge-sharing among regenerative farmers and researchers globally
- Tailoring regenerative practices to local contexts and coupling them with other sustainable food system strategies like reducing waste and shifting diets

Importantly, scaling regenerative agriculture must prioritize the leadership and rights of Indigenous and local communities, ensuring they have access to land, resources, and decision-making power. A just and inclusive approach is essential for regenerative agriculture to achieve its full potential in nourishing people and the planet.